In [1]:
# use following link for tutorial
# https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/


In [18]:
# 1. Handle Data

#load Pima indians dataset
import pandas as pd

filename = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
dataset = pd.read_csv(filename);
print(f'Loaded data file {filename} with {len(dataset)} rows');


Loaded data file https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv with 767 rows
767


In [20]:
import random
def splitData(dataset, splitRatio):
    trainSize = int(len(dataset)*splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

dataset = [[1], [2], [3], [4], [5]]
splitRatio = 0.67
train, test = splitData(dataset, splitRatio)
print(f'Split {len(dataset)} into train with {train} and test with {test}')


Split 5 into train with [[4], [1], [2]] and test with [[3], [5]]
